In [4]:
import pandas as pd
import numpy as np
import math
from operator import itemgetter
import matplotlib.pyplot as plt

#Función para calcular el tiempo con un modelo exponencial

def calcula_tiempo(tasa):
    random_number = np.random.rand(1)
    tiempo = (-1/tasa)*math.log(random_number) 


    return tiempo
    

In [5]:
#Función para crear un DataFrame a partir de los archivos entregados para las rutas de cada red
#Los archivos se encuentran en una subcarpeta de nombre 'rutas'
N = 0
L = 0

def create_df(file_name):
    df = pd.read_csv('./rutas/'+file_name,header=None)
    df = df[0].str.split('\t', expand=True)
    N = int(df[3][1])
    L = int(df[3][2])
    return df, N, L


def clean_df_and_C(df):
    df = df[df[0] != df[1]][6:].reset_index(drop=True)
    C = df.iloc[:,3:].apply(pd.value_counts).sum(axis=1).max()
    df['rutas'] = df.iloc[:,3:].apply(
            lambda x: [i for i in x.dropna().astype(int)],
            axis=1
        )
    return df, C
df, N, L = create_df('EuroCore-WDM.rut')
df, C = clean_df_and_C(df)
M = N*(N-1)
C = int(C/2)
display(df)
print(C)

,0,1,2,3,4,5,rutas
0,0,1,1,0,None,None,[0]
1,0,1,2,6,14,None,"[6, 14]"
2,0,2,2,0,8,None,"[0, 8]"
3,0,2,2,6,18,None,"[6, 18]"
4,0,3,3,6,19,16,"[6, 19, 16]"
...,...,...,...,...,...,...,...
215,10,7,3,41,5,6,"[41, 5, 6]"
216,10,8,2,49,47,None,"[49, 47]"
217,10,8,2,27,24,None,"[27, 24]"
218,10,9,1,49,None,None,[49]


6


In [6]:
def create_FEL(usuarios,df,M):
    t_off = 20
    FEL = []
    i = 0
    while (i < M):
        print(df['rutas'][i])
        FEL.append(['T',i,'OFF',calcula_tiempo(t_off),df['rutas'][i],df['rutas'][i+1]])
        i += 2
    #for i in df.index:
    #    FEL.append(['T',i,'OFF',calcula_tiempo(t_off),df['rutas'][i]])
    
    for x in range(usuarios):
        FEL.append(['F',x,'OK',calcula_tiempo(t_off)])

    FEL.sort(key=itemgetter(3))

    return FEL

FEL = create_FEL(49,df,220)
FEL



[0]
[0, 8]
[6, 19, 16]
[2, 29]
[2]
[4]
[6]
[6, 42]
[4, 38]
[4, 40]
[1]
[8]
[10, 23]
[10]
[1, 2]
[12]
[14]
[8, 20]
[14, 44]
[10, 32]
[9, 1]
[9]
[16]
[20, 31]
[16, 26, 37]
[9, 12]
[18]
[20]
[18, 44]
[16, 26]
[17, 18, 7]
[22, 11]
[17]
[22]
[22, 28]
[26, 41]
[24, 43]
[24]
[24, 46]
[26]
[28, 3]
[11]
[30, 21]
[23]
[28]
[32, 41]
[11, 14]
[30]
[32, 49]
[32]
[3]
[3, 0]
[36, 27, 17]
[29, 23]
[29]
[34]
[3, 6]
[29, 30]
[34, 38]
[36]
[5]
[13]
[13, 8]
[40, 27]
[40, 33]
[35]
[13, 14]
[38, 47]
[38]
[40]
[7]
[15]
[19]
[42, 25]
[15, 10]
[7, 2]
[15, 12]
[42]
[44]
[44, 48]
[43, 7]
[21, 9]
[21]
[25]
[31]
[31, 28]
[46, 39]
[43]
[46]
[46, 48]
[39, 5]
[45, 15]
[45, 19]
[47, 25]
[48, 33]
[39, 35]
[39]
[45]
[47]
[48]
[41, 5]
[33, 11]
[27, 17]
[27]
[33]
[37]
[41]
[49, 45]
[49, 47]
[49]


[['F', 35, 'OK', 0.00040519839500181517],
 ['F', 14, 'OK', 0.0005646824777088945],
 ['T', 80, 'OFF', 0.001028242940772797, [28, 3], [11, 1]],
 ['F', 1, 'OK', 0.0014044199964266414],
 ['F', 13, 'OK', 0.0018511883158222862],
 ['T', 100, 'OFF', 0.0020955019933064974, [3], [34, 5]],
 ['F', 39, 'OK', 0.0024589647159849507],
 ['T', 192, 'OFF', 0.0025411508563487175, [39], [48, 41]],
 ['T', 36, 'OFF', 0.0030888549310234405, [14, 44], [12, 38]],
 ['T', 154, 'OFF', 0.004219284350373924, [42], [19, 20]],
 ['T', 130, 'OFF', 0.0043156065241462, [35], [40, 37]],
 ['T', 124, 'OFF', 0.004720747941886458, [13, 8], [40, 27, 17]],
 ['T', 106, 'OFF', 0.004950249910343475, [29, 23], [36, 27]],
 ['T', 22, 'OFF', 0.004986514991611932, [8], [14, 19]],
 ['T', 120, 'OFF', 0.005012250230116457, [5], [13, 1]],
 ['T', 132, 'OFF', 0.005411595306485568, [13, 14], [38, 45]],
 ['T', 168, 'OFF', 0.005477757855358685, [31], [25, 22]],
 ['T', 86, 'OFF', 0.005565566768183452, [23], [30, 25]],
 ['F', 28, 'OK', 0.005583997

In [27]:
def simulator(FEL,C):
    t_off = 20
    t_on = 1
    t_falla = 20
    LLEGADAS = math.pow(10,3)
    uso_rutas_secundarias = 0
    bloqueos = 0
    bloqueo_capacidad = 0
    fallas_ocurridas = 0
    transmisiones = 0
    enlace_fallido = 'ninguno'
    FEL_sim = FEL
    P = 1
    C_sim = C
    falla = []
    while (transmisiones <= 50):
        
        evento_actual = FEL_sim.pop(0)                  #obtenemos el evento que viene en el tiempo siguiente

        if (evento_actual[0] == 'F'):                   #si es una falla entra aqui

            falla = evento_actual                
            fallas_ocurridas += 1
                                                 
            falla[3] = falla[3] + calcula_tiempo(t_falla)   #sumamos tiempo para que esta falla vuelva a ocurrir mas adelante
                                                
            enlace_fallido = falla[1]
            
            FEL_sim.append(falla)                   #agregamos la falla de nuevo a la FEL de eventos con un tiempo mayor
            print('Enlace con falla: ',enlace_fallido)
            
        if (evento_actual[0] == 'T'):               #si es una transmision entra aqui
            
            if (evento_actual[2] == 'OFF'):         #si esta en OFF, es porque va a transmitir

                transmisiones += 1
                ruta_primaria = evento_actual[4]        #obtenemos la ruta primaria que usara la transmision
                ruta_secundaria = evento_actual[5]      #obtenemos la ruta secundaria que usara la transmision
                print('Ruta primaria: ',ruta_primaria)
                print('Ruta secundaria: ',ruta_secundaria)
                if (enlace_fallido in ruta_primaria):   #si el enlace que esta con falla esta presente en la ruta primaria entra aqui
                    uso_rutas_secundarias += 1

                    if (enlace_fallido in ruta_secundaria): #si el enlace que esta con falla tambien esta presente en la ruta secundaria entonces no hay transmision
                        bloqueos += 1

                    #comienza proceso de transmision por la ruta secundaria
                    else:
                        if (C_sim > 0):             #verificamos si hay long de onda disponibles en la red para transmitir
                            C_sim -= 1              #restamos 1 a la capacidad de transmision de la red, debido a que se tendran long de onda limitadas
                            evento_actual[3] = evento_actual[3] + calcula_tiempo(t_on)
                            evento_actual[2] = 'ON'
                            FEL_sim.append(evento_actual)

                        else:
                            bloqueo_capacidad += 1
                            evento_actual[3] = evento_actual[3] + calcula_tiempo(t_off)
                            FEL_sim.append(evento_actual)
                #comienza proceso de transmision por la ruta primaria            
                else:
                    if (C_sim > 0):
                        C_sim -= 1
                        evento_actual[3] = evento_actual[3] + calcula_tiempo(t_on)
                        evento_actual[2] = 'ON'
                        FEL_sim.append(evento_actual)

                    else:
                        bloqueo_capacidad += 1
                        evento_actual[3] = evento_actual[3] + calcula_tiempo(t_off)
                        FEL_sim.append(evento_actual)
            elif (evento_actual[2] == 'ON'):        #si esta en ON, es porque terminará de transmitir
                C_sim += 1
                evento_actual[3] = evento_actual[3] + calcula_tiempo(t_off)
                evento_actual[2] = 'OFF'
                FEL_sim.append(evento_actual)

        #FEL_sim.append(evento_actual)
        FEL_sim.sort(key=itemgetter(3))

    print('Bloqueos: ',bloqueo_capacidad)
    print('Uso de rutas secundarias: ', uso_rutas_secundarias, '\n')
    return 'succes'


simulator(FEL,C)

Ruta primaria:  [43]
Ruta secundaria:  [21, 18]
Ruta primaria:  [32]
Ruta secundaria:  [28, 36]
Ruta primaria:  [40]
Ruta secundaria:  [35, 36]
Enlace con falla:  12
Enlace con falla:  7
Enlace con falla:  7
Ruta primaria:  [31]
Ruta secundaria:  [25, 22]
Ruta primaria:  [8]
Ruta secundaria:  [14, 19]
Enlace con falla:  20
Enlace con falla:  27
Ruta primaria:  [20]
Ruta secundaria:  [18, 42]
Ruta primaria:  [39, 5]
Ruta secundaria:  [45, 7]
Enlace con falla:  31
Ruta primaria:  [11, 14]
Ruta secundaria:  [30, 43]
Ruta primaria:  [21]
Ruta secundaria:  [25, 17]
Enlace con falla:  48
Ruta primaria:  [48, 33]
Ruta secundaria:  [47, 31]
Enlace con falla:  25
Ruta primaria:  [24, 43]
Ruta secundaria:  [17, 18]
Ruta primaria:  [3, 0]
Ruta secundaria:  [34, 13]
Ruta primaria:  [34, 38]
Ruta secundaria:  [36, 49]
Enlace con falla:  43
Ruta primaria:  [48]
Ruta secundaria:  [39, 40]
Enlace con falla:  29
Enlace con falla:  13
Enlace con falla:  45
Ruta primaria:  [9, 12]
Ruta secundaria:  [16, 

'succes'

In [7]:
test = set([1,2,3,4,5])
test2 = set([2,4])
test3 = [1,2,3,4,5]
if (test2.issubset(test)):
    print("la contiene")

if 3 in test3:
    print('esta en el array')

la contiene
esta en el array
